# Segmenting and Clustering Neighborhoods in Toronto

### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Install BeautifulSoup and import library to parse HTML and XML documents

In [2]:
# !pip install beautifulsoup4
# !pip install lxml
# !pip install html5lib

from bs4 import BeautifulSoup


### Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
# import library to open URLs
import urllib.request

# specify URL for scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open url and put into the page variable
page = urllib.request.urlopen(url)

# parse HTML into BeautifulSoup
#soup = BeautifulSoup(page, "lxml")
soup = BeautifulSoup(page, "html.parser")


### Print the content to check where the data is

In [23]:
# check result to find the data
# print (soup.prettify())

### Looks like the content we're looking for is surrounded by the table tag with class = "wikitable sortable"
### Isolate the data

In [4]:
# the table is surrounded by the tag <table class="wikitable sortable">
right_table=soup.find('table', class_='wikitable sortable')
#right_table


### Get the table content and put them in lists

Each row is surrounded by <tr> tags. Each column is surrounded by <td> tags

In [5]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        

### Create Data Frame from the lists above
### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
df = pd.DataFrame(A, columns=['Postal Code'])
df['Borough'] = B
df['Neighborhood'] = C

df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


### Clean up the data

### Ignore cells with a borough that is Not assigned

In [9]:
# Ignore cells with a borough that is Not assigned
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)



### More than one neighborhood can exist in one postal code area. 
### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
### These two rows will be combined into one row with the neighborhoods separated with a comma.

In [10]:
# remove the \n
df = df.replace({'\n':''}, regex=True)

# replace the / with ,
df['Neighborhood'] = df['Neighborhood'].replace({'/':','}, regex=True)

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### Use the .shape method to print the number of rows of your dataframe

In [11]:
df.shape

(103, 3)

### Download Geospatial Data csv file

In [12]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the Neighborhood dataframe with the Geo Data dataframe to get Latitude and Longitude by Neighborhood

In [13]:
# check to see how many rows of data in the neighborhood dataframe
df.shape

(103, 3)

In [14]:
# check to see how many rows of data in the geo dataframe
geo.shape

(103, 3)

In [15]:
# merging the neighborhood dataframe with geo data dataframe
df_toronto = pd.merge(df, geo, on='Postal Code')
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(df_toronto, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare Credentials

In [18]:
CLIENT_ID = 'GKJTPRRR2GAKPOACEXGOYDD0JBLXHXIPU1VJBHKNMKUWI5LY' # your Foursquare ID
CLIENT_SECRET = 'YLTY20AFU0MMDNL23V1VRSQEN5YOPORSZPG4EMT2SVPMJTPM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GKJTPRRR2GAKPOACEXGOYDD0JBLXHXIPU1VJBHKNMKUWI5LY
CLIENT_SECRET:YLTY20AFU0MMDNL23V1VRSQEN5YOPORSZPG4EMT2SVPMJTPM


### Downtown Toronto

In [19]:
# get neighborhood in downtown Toronto
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Function to pull top 100 venues within 500 meters of each neighborhood in Downtown Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the function above for the neighborhoods in Downtown Toronto

In [26]:
LIMIT = 100
downtown_venues = getNearbyVenues(names = downtown_data['Neighborhood'],
                                   latitudes = downtown_data['Latitude'],
                                   longitudes = downtown_data['Longitude']
                                  )


Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst
Rosedale
Stn A PO Boxes
St. James Town , Cabbagetown
First Canadian Place , Underground city
Church and Wellesley


### Check the dataframe

In [27]:
print(downtown_venues.shape)
downtown_venues.head()

(1235, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### Analyze each neighbodhood

In [28]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.00,0.000000,0.000000,0.000000,0.000000,0.017857,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.053571,0.000000,0.000000,0.000000,0.00,0.000000,0.017857,0.000000,0.017857,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.000000,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017857,0.00,0.00,0.000000,0.017857,0.00,0.000000,0.017857,0.035714,0.017857,0.017857,0.000000,0.017857,0.000000,0.00,0.000000,0.

### Function to sort venues in descending order

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood

In [55]:
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Beer Bar,Juice Bar,Park,Creperie,Hotel,Concert Hall,Eastern European Restaurant,Comfort Food Restaurant,Lounge,Breakfast Spot,Indian Restaurant,Irish Pub,Museum,Butcher,Liquor Store,Nightclub,Jazz Club,Diner,Japanese Restaurant,Bistro,Art Gallery,Steakhouse,Vegetarian / Vegan Restaurant,French Restaurant,Fountain,Gourmet Shop,Greek Restaurant,Thai Restaurant,Shopping Mall,Tailor Shop,Fish Market,BBQ Joint,Cosmetics Shop,Bagel Shop,Beach,Basketball Stadium,Doner Restaurant,Dessert Shop,Gaming Cafe,Cupcake Shop,Furniture / Home Store
1,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Coffee Shop,Airport Terminal,Rental Car Location,Boat or Ferry,Sculpture Garden,Boutique,Harbor / Marina,Airport Lounge,Airport Gate,Airport Food Court,Airport,Bar,Dog Run,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Diner,Distribution Center,Discount Store,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Creperie,Cosmetics Shop,Concert Hall,Falafel Restaurant,Fish Market,Fast Food Restaurant,Filipino Restaurant,Gourmet Shop,Gluten-free Restaurant,Gift Shop,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Sushi Restaurant,Japanese Restaurant,Spa,Ice Cream Shop,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Indian Restaurant,Park,Office,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Korean Restaurant,Bar,Chinese Restaurant,Dessert Shop,Hotel,Gastropub,Furniture / Home Store,Comic Shop,French Restaurant,Falafel Restaurant,Donut Shop,Discount Store,Diner,Gym / Fitness Center,Yoga Studio,Thai Restaurant,Wine Bar,Vegetarian / Vegan Restaurant,Steakhouse,Comfort Food Restaurant,Dance Studio,Cupcake Shop,Creperie,Cosmetics Shop,Concert Hall,Theme Restaurant,Colombian Restaurant,Theater,College Rec Center,College Gym,College Auditorium,College Arts Building
3,Christie,Grocery Store,Café,Park,Gas Station,Italian Restaurant,Restaurant,Diner,Baby Store,Candy Store,Athletics & Sports,Coffee Shop,Nightclub,Comic Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Cupcake Shop,Dance Studio,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Electronics Store,Department Store,Deli / Bodega,Eastern European Restaurant,Falafel Restaurant,Ethiopian Restaurant,Fried Chicken Joint,Gourmet Shop,Gluten-free Restaurant,Gift Shop,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub,Gaming Cafe,Furniture / Ho

### Run k-means to cluster the neighborhood into 5 clusters

In [56]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 4, 3, 1, 1, 1, 1, 1, 1], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,4,Coffee Shop,Pub,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Theater,Hotel,Wine Shop,Health Food Store,Historic Site,Gym / Fitness Center,Ice Cream Shop,Farmers Market,Performing Arts Venue,French Restaurant,Distribution Center,Event Space,Electronics Store,Shoe Store,Dessert Shop,Cosmetics Shop,Chocolate Shop,Beer Store,Bank,Art Gallery,Antique Shop,Restaurant,Yoga Studio,Spa,Seafood Restaurant,Comfort Food Restaurant,Deli / Bodega,Dance Studio,Cupcake Shop,Creperie,Tanning Salon,Concert Hall,Comic Shop,Colombian Restaurant,Taiwanese Restaurant,College Rec Center,College Gym,College Auditorium,College Arts Building,Tea Room,Cocktail Bar,Clothing Store,Church
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Diner,Gym,Hobby Shop,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,Park,College Auditorium,Creperie,Bank,Mexican Restaurant,Discount Store,Distribution Center,Juice Bar,Italian Restaurant,Fried Chicken Joint,Bar,Yoga Studio,Theater,Arts & Crafts Store,Bakery,Dumpling Restaurant,Farmers Market,Falafel Restaurant,Event Space,Airport Service,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Dance Studio,Doner Restaurant,Dog Run,Airport Terminal,American Restaurant,Antique Shop,Dessert Shop,Department Store,Airport Lounge,Fast Food Restaurant,Filipino Restaurant,Fish Market,German Restaurant,Airport,General Travel,General Entertainment,Gay Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Diner,Bookstore,Fast Food Restaurant,Lingerie Store,Plaza,Restaurant,Theater,Electronics Store,Tea Room,Ramen Restaurant,Burger Joint,Office,Miscellaneous Shop,Pizza Place,Other Great Outdoors,Movie Theater,Burrito Place,Park,Modern European Restaurant,Gym / Fitness Center,Mexican Restaurant,Lounge,Chinese Restaurant,Poutine Place,College Rec Center,Comic Shop,Hotel,Department Store,Dessert Shop,Gym,Ethiopian Restaurant,Gastropub,Furniture / Home Store,Lake,Music Venue,Pub,Bank,Wine Bar,Vietnamese Restaurant,Video Game Store,Thai Restaurant,Tanning Salon
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Park,Seafood Restaurant,Moroccan Restaurant,Cosmetics Shop,Lingerie Store,Breakfast Spot,Diner,Japanese Restaurant,Department Store,Farmers Market,Gym,Gastropub,Irish Pub,Clothing Store,Burrito Place,Pharmacy,French Restaurant,Cheese Shop,Optical Shop,Fountain,German Restaurant,Food Truck,Ice Cream Shop,Food & Drink Shop,Grocery St

### Visualize the resulting clusters

In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Cluster 1

In [59]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
13,Downtown Toronto,0,Airport Service,Coffee Shop,Airport Terminal,Rental Car Location,Boat or Ferry,Sculpture Garden,Boutique,Harbor / Marina,Airport Lounge,Airport Gate,Airport Food Court,Airport,Bar,Dog Run,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Diner,Distribution Center,Discount Store,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Creperie,Cosmetics Shop,Concert Hall,Falafel Restaurant,Fish Market,Fast Food Restaurant,Filipino Restaurant,Gourmet Shop,Gluten-free Restaurant,Gift Shop,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant


### Examine Cluster 2

In [60]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Diner,Bookstore,Fast Food Restaurant,Lingerie Store,Plaza,Restaurant,Theater,Electronics Store,Tea Room,Ramen Restaurant,Burger Joint,Office,Miscellaneous Shop,Pizza Place,Other Great Outdoors,Movie Theater,Burrito Place,Park,Modern European Restaurant,Gym / Fitness Center,Mexican Restaurant,Lounge,Chinese Restaurant,Poutine Place,College Rec Center,Comic Shop,Hotel,Department Store,Dessert Shop,Gym,Ethiopian Restaurant,Gastropub,Furniture / Home Store,Lake,Music Venue,Pub,Bank,Wine Bar,Vietnamese Restaurant,Video Game Store,Thai Restaurant,Tanning Salon
3,Downtown Toronto,1,Café,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Park,Seafood Restaurant,Moroccan Restaurant,Cosmetics Shop,Lingerie Store,Breakfast Spot,Diner,Japanese Restaurant,Department Store,Farmers Market,Gym,Gastropub,Irish Pub,Clothing Store,Burrito Place,Pharmacy,French Restaurant,Cheese Shop,Optical Shop,Fountain,German Restaurant,Food Truck,Ice Cream Shop,Food & Drink Shop,Grocery Store,Comfort Food Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Creperie,Hostel,Church,Furniture / Home Store,Thai Restaurant,Bistro,Tailor Shop,Bakery,Belgian Restaurant,Salon / Barbershop,Asian Restaurant,BBQ Joint,Vegetarian / Vegan Restaurant,Bookstore
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Beer Bar,Juice Bar,Park,Creperie,Hotel,Concert Hall,Eastern European Restaurant,Comfort Food Restaurant,Lounge,Breakfast Spot,Indian Restaurant,Irish Pub,Museum,Butcher,Liquor Store,Nightclub,Jazz Club,Diner,Japanese Restaurant,Bistro,Art Gallery,Steakhouse,Vegetarian / Vegan Restaurant,French Restaurant,Fountain,Gourmet Shop,Greek Restaurant,Thai Restaurant,Shopping Mall,Tailor Shop,Fish Market,BBQ Joint,Cosmetics Shop,Bagel Shop,Beach,Basketball Stadium,Doner Restaurant,Dessert Shop,Gaming Cafe,Cupcake Shop,Furniture / Home Store
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Hotel,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Steakhouse,Clothing Store,Breakfast Spot,Bookstore,Bar,Salad Place,Pizza Place,Asian Restaurant,American Restaurant,Concert Hall,Gluten-free Restaurant,Gift Shop,Ice Cream Shop,Women's Store,Japanese Restaurant,Gastropub,Juice Bar,Latin American Restaurant,Lounge,Mediterranean Restaurant,Modern European Restaurant,New American Restaurant,General Travel,Department Store,Furniture / Home Store,Burrito Place,Art Gallery,Bakery,Bank,Brazilian Restaurant,Building,Burger Joint,Colombian Restaurant,Food Court,Cupcake Shop,Park,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Nightclub
8,Downtown T

### Examine Cluster 3

In [61]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
14,Downtown Toronto,2,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Dance Studio,Ethiopian Restaurant,Cupcake Shop,Creperie,Cosmetics Shop,Concert Hall,Comic Shop,Comfort Food Restaurant,Colombian Restaurant,College Rec Center,College Gym,Electronics Store,Event Space,College Arts Building,Furniture / Home Store,Gourmet Shop,Gluten-free Restaurant,Gift Shop,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub,Gas Station,Gaming Cafe,Fried Chicken Joint,Falafel Restaurant,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fish Market,Filipino Restaurant,Fast Food Restaurant


### Examine Cluster 4

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]